In [1]:
print("Hello, World!")

Hello, World!


In [2]:
%pwd

'c:\\Users\\chapt\\OneDrive\\Desktop\\pregchatbot\\End-to-End-Pregwell-Chatbot-Generative-AI\\research'

In [3]:
import os
os.chdir("../") 

In [4]:
%pwd

'c:\\Users\\chapt\\OneDrive\\Desktop\\pregchatbot\\End-to-End-Pregwell-Chatbot-Generative-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data from PDF File
# Tabnine: Edit | Test | Explain | Document | Ask
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader,)
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data="Data/")

In [9]:
#extracted_data   

In [10]:
#Split the Data into Text Chunks
#Tabnine: Edit | Test | Explain | Document | Ask
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunks= text_splitter.split_documents(extracted_data)

    return texts_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1270


In [13]:
#text_chunks

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
#Download the Embeddings from HuggingFace
#Tabnine: Edit | Test | Explain | Document | Ask
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
    )
    return embeddings

In [19]:
embeddings = download_hugging_face_embeddings()

C:\Users\chapt\AppData\Local\Temp\ipykernel_24036\2758542165.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\chapt\miniconda3\envs\pregwell\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
query_results = embeddings.embed_query("Hello world")
print("Length",len(query_results))

Length 384


In [22]:
#query_results  

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [28]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

# pc = Pinecone)  # Replace with your actual Pinecone API key

index_name = "pregwellbot"

# pc.create_index(
#     name=index_name,
#     dimension=384,  # Dimension of the embeddings
#     metric="cosine",  # Similarity metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1",  # Replace with your desired region
#     )
# )

In [29]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY  

In [30]:
#embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [33]:
# load Existing Index

from langchain_pinecone import PineconeVectorStore
# Embed eqch chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [34]:
docsearch 

In [35]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,  # Number of similar documents to retrieve
    }
)


In [36]:
retrived_docs = retriever.invoke("What is Hepatitis B?")

In [37]:
retrived_docs

[Document(id='b9287bb3-f752-4594-8eac-d8274c2ad239', metadata={'creationdate': '2013-11-25T12:51:17+00:00', 'creator': 'PyPDF', 'moddate': '2013-11-25T12:51:17+00:00', 'page': 38.0, 'page_label': '37', 'producer': 'PyPDF', 'source': 'Data\\Pregnancy_Book_comp.pdf', 'total_pages': 196.0}, page_content='Hepatitis B\nHepatitis B is a virus that infects the \nliver. Many people with hepatitis B, \nwill have no signs of illness, but \nthey might infect others. If you \nhave hepatitis B, or are infected \nduring pregnancy, you can pass \nthe infection to your baby at birth. \nYou will be offered a blood test for \nhepatitis B as part of your antenatal \ncare (see page 47). Babies who \nare at risk should be immunised \nat birth. This is 90–95% effective \nin preventing them from getting'),
 Document(id='61fafaab-b020-4982-a5d2-cad584ddabad', metadata={'creationdate': '2013-11-25T12:51:17+00:00', 'creator': 'PyPDF', 'moddate': '2013-11-25T12:51:17+00:00', 'page': 38.0, 'page_label': '37', 'pr

In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",           # or 
    temperature=0.4,
    max_tokens=500
)


In [51]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a helpful assistant specialized in answering questions strictly related to health and pregnancy. "
    "Use the following retrieved context to answer the user's question. "
    "If the question is not related to pregnancy or maternal health, respond with: "
    "'Apologies, but I can only provide assistance on matters concerning health and pregnancy.' "
    "Keep your answers concise and limited to a maximum of three sentences.\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [53]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(
    retriever, question_answer_chain
)

In [49]:
response = rag_chain.invoke({"input": "What is Hepatitis B?"})
print(response["answer"]) 

Hepatitis B is a virus that infects the liver. Many people with hepatitis B may show no signs of illness but can still infect others. It can be transmitted from an infected mother to her baby at birth, and babies at risk should be immunized at birth to prevent infection.


In [54]:
response = rag_chain.invoke({"input": "What is chelsea?"})
print(response["answer"]) 

Apologies, but I can only provide assistance on matters concerning health and pregnancy.
